In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException


driver = webdriver.Chrome()
driver.get('https://www.justeat.it/area/00185-roma')

In [13]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight-100)")

In [14]:
result = driver.find_elements(By.CLASS_NAME, 'c-listing-loader')
array = result[0].find_elements(By.CLASS_NAME, 'c-restaurantCardContainer')
len(array)

349

In [15]:
urls = []
for i in array:
    urls.append(i.get_attribute('href'))

len(urls)
ristorantidacercare=100 #Imposta Quanti ristoranti vuoi Cercare.
urls = urls[0:ristorantidacercare] 
urls

['https://www.justeat.it/restaurants-sfizio-pizzeriae-gastronomia-roma/menu',
 'https://www.justeat.it/restaurants-sunnyroma/menu',
 'https://www.justeat.it/restaurants-yesfoodpiazzabologna/menu',
 'https://www.justeat.it/restaurants-leyuang/menu',
 'https://www.justeat.it/restaurants-gelateria-farini-roma/menu',
 'https://www.justeat.it/restaurants-ami-poke---hawaiian-bar---re-di-roma-00183/menu',
 'https://www.justeat.it/restaurants-amipokehawaiianbar/menu',
 'https://www.justeat.it/restaurants-sushiinthebox/menu',
 'https://www.justeat.it/restaurants-poke-poke-roma-00199/menu',
 'https://www.justeat.it/restaurants-freetto---termini-roma/menu',
 'https://www.justeat.it/restaurants-mcdonalds-rediroma/menu',
 'https://www.justeat.it/restaurants-sushikotopo/menu',
 'https://www.justeat.it/restaurants-yesfoodasioamericanasanlorenzoroma/menu',
 'https://www.justeat.it/restaurants-chiyosushirestaurant/menu',
 'https://www.justeat.it/restaurants-ristorantexun-roma/menu',
 'https://www.juste

In [ ]:
dishes = webdriver.Chrome()
count = len(urls)
result=[]
piatto={"nome":"",
        "prezzo":"",
        "descrizione":""}
i=0

ristorante={
    "nome":"",
    "indirizzo":"",
    "rating":"",
    "piatti":[]  
}

portate=[];
for obj in urls:
    #Entra in Ogni Link estratto
    link=urls[i]
    dishes.get(link)
    
    
    #Trova il Nome , Indirizzo e Rating del Ristorante
    ristorante["nome"]=dishes.find_elements(By.TAG_NAME,"h1")[0].text
    ristorante["indirizzo"]=dishes.find_elements(By.CLASS_NAME,"c-restaurantHeader-address-content")[0].text
    ristorante["rating"]= dishes.find_elements(By.CLASS_NAME,"c-rating")[0].find_elements(By.XPATH , "./*[1]")[0].get_attribute("class") 
    ristorante["rating"]=ristorante["rating"][-2] + ristorante["rating"][-1] #Tronca il Tag Class per trovare il Rating
    
    print(f'<> Ristorante Numero: {len(urls)} --- Nome Ristorante > {ristorante["nome"]} --- Mancanti > {ristorantidacercare-i}')
    
    #Lista delle Portate
    portate=dishes.find_elements(By.CLASS_NAME, 'c-menuItems-content')

    j=0

    for piatti in portate: #Ciclo for per Scorrere tutti i piatti di un Ristorante(Aggiungere Le Categorie)
        
        #Trova i Nomi dei Piatti
        piatto["nome"]=dishes.find_elements(By.CLASS_NAME, 'c-menuItems-content')[j].find_elements(By.CLASS_NAME, 'c-menuItems-heading.u-spacingRight')[0].text
        
        #Trova i Prezzi dei Piatti (If else scritto perchè a volte i prezzi sono visualizzati con due classname diversi )
        if (dishes.find_elements(By.CLASS_NAME, 'c-menuItems-content')[j].find_elements(By.XPATH, '//*[@data-js-test="menu-item-price"]')[j].text != "") :
            piatto["prezzo"]=dishes.find_elements(By.CLASS_NAME, 'c-menuItems-content')[j].find_elements(By.XPATH, '//*[@data-js-test="menu-item-price"]')[j].text
        else :
            #Questo codice non funziona se viene aperta (a caso) la schermata con i bottoni al posto dei prezzi
            piatto["prezzo"]=dishes.find_elements(By.CLASS_NAME, 'c-menuItems-content')[j].find_elements(By.CLASS_NAME, 'u-spacingTop.Button_o-btn_1KX8u.Button_o-btn--secondary_1KBR1.Button_o-btn--sizeXSmall_3KRtq.c-carousel-slide.notranslate')[0].text
 
        #Trova le descrizioni dei Piatti (Se Esistenti)
        try:
            piatto["descrizione"]=dishes.find_elements(By.CLASS_NAME, 'c-menuItems-content')[j].find_elements(By.TAG_NAME, 'p')[0].text
        except NoSuchElementException:
            piatto["descrizione"]=""
        
        if piatto["prezzo"]==piatto["descrizione"] :
            piatto["descrizione"]=""
        
        ristorante["piatti"].append(piatto)
        
        piatto={"nome":"",
        "prezzo":"",
        "descrizione":""}
        j+=1
    
    result.append(ristorante)
    
    ristorante={
        "nome":"",
        "indirizzo":"",
        "rating":"",
        "piatti":[]
        }
    
 
    i+=1



In [6]:
import json
out_file = open("../data/restaurants.json", "w")
for x in result :
    
    json_data = json.dump(x, out_file, indent=4)


In [37]:
import json
jsonRist = json.load(open("../data/restaurants.json"))

In [38]:
for indxRis in range(len(jsonRist)):
    addr = jsonRist[indxRis]["address"].split(", ")
    zipcode = addr[len(addr)-1]
    address = ""

    for indStr in range(len(addr)-1):
        if indStr == len(addr)-2:
            address += addr[indStr]
            break
        address += addr[indStr] + ", "

    jsonRist[indxRis]["address"] = address
    jsonRist[indxRis]["zipcode"] = zipcode

In [39]:
with open("../data/restaurantsZip.json", "w") as outFile:
    json.dump(jsonRist, outFile)